In [2]:
from keras.layers import Dense, Dropout, LSTM, Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential

C:\Users\konya\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
import pandas as pd
import numpy as np

In [30]:
train_data = pd.read_csv("C:\\Users\\konya\\Desktop\\DNA TFBP\\DNA_TranscriptionFactorBindingPrediction\\data\\train.csv");
test_data = pd.read_csv("C:\\Users\\konya\\Desktop\\DNA TFBP\\DNA_TranscriptionFactorBindingPrediction\\data\\test.csv");
train_len = int(len(train_data))
print(train_len)

2000


In [110]:
#split train data into train and validation data
split_ratio = 0.8

train_size = int( train_len * (split_ratio))

#before taking partitions shuffle rows
#this is because all same class tuples are grouped together
train_data = train_data.sample(frac=1).reset_index(drop=False)

X_train = np.array(train_data['sequence'][0:])
Y_train = np.array(train_data['label'][0:])
X_test = np.array(train_data['sequence'][train_size:])
Y_test = np.array(train_data['label'][train_size:])


In [146]:
#preprocessing of DNA sequences
from keras.preprocessing.text import Tokenizer
from keras import optimizers

tokenizer = Tokenizer(split='', char_level=True)
tokenizer.fit_on_texts(X_train)

print(tokenizer.word_counts)
print(tokenizer.document_count)
print(tokenizer.word_index)
print(tokenizer.word_docs)

encoded_X_train = tokenizer.texts_to_sequences(X_train) #mode for text_to_matrix = freq,count,binary,tfidf
print(encoded_X_train[0])

OrderedDict([('C', 6904), ('T', 5713), ('G', 9361), ('A', 6022)])
2000
{'G': 1, 'C': 2, 'A': 3, 'T': 4}
{'T': 1710, 'A': 1787, 'G': 1905, 'C': 1899}
[2, 2, 4, 1, 2, 4, 1, 2, 3, 2, 4, 1, 2, 4]


In [153]:
#LSTM model in keras
def model_init(input_length):
    LSTM_model = Sequential()
    print(input_length)
    LSTM_model.add(Embedding(5,100, input_length = 14))
    
    LSTM_model.add(LSTM(activation="sigmoid", recurrent_activation="hard_sigmoid", return_sequences=True, units=256))
    LSTM_model.add(LSTM(activation="sigmoid", recurrent_activation="hard_sigmoid", units=256))
    LSTM_model.add(Dense(1, activation='sigmoid'))

    LSTM_model.compile(loss='binary_crossentropy', optimizer='adagrad', metrics=['accuracy'])
    return LSTM_model

In [124]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(X_test.shape)

(2000,)
(2000,)
(400,)
(400,)


In [154]:
model = model_init(train_len)

trained_model = model.fit(np.array(encoded_X_train), np.array(Y_train), batch_size=10, epochs=15, validation_split = 0.2, verbose = 1)


2000
Train on 1600 samples, validate on 400 samples
Epoch 1/15
1600/1600 [==============================] - 27s 17ms/step - loss: 0.5895 - acc: 0.6788 - val_loss: 0.4075 - val_acc: 0.8425
Epoch 2/15
1600/1600 [==============================] - 23s 14ms/step - loss: 0.3591 - acc: 0.8456 - val_loss: 0.3550 - val_acc: 0.8500
Epoch 3/15
1600/1600 [==============================] - 23s 14ms/step - loss: 0.3363 - acc: 0.8519 - val_loss: 0.3779 - val_acc: 0.8450
Epoch 4/15
1600/1600 [==============================] - 22s 14ms/step - loss: 0.3306 - acc: 0.8619 - val_loss: 0.3388 - val_acc: 0.8625
Epoch 5/15
1600/1600 [==============================] - 22s 14ms/step - loss: 0.3240 - acc: 0.8550 - val_loss: 0.3520 - val_acc: 0.8625
Epoch 6/15
1600/1600 [==============================] - 22s 14ms/step - loss: 0.3267 - acc: 0.8600 - val_loss: 0.3595 - val_acc: 0.8625
Epoch 7/15
1600/1600 [==============================] - 23s 15ms/step - loss: 0.3249 - acc: 0.8600 - val_loss: 0.3424 - val_acc: 0.8

In [155]:
encoded_X_test = tokenizer.texts_to_sequences(X_test)
score, accuracy = model.evaluate(np.array(encoded_X_test), np.array(Y_test), batch_size=1)
print('Score:', score)
print('Accuracy:', accuracy)

400/400 [==============================] - 3s 8ms/step
Score: 0.33581819218583403
Accuracy: 0.865


In [160]:
from keras.models import model_from_json

model_JSON = model.to_json()
with open("C:\\Users\\konya\\Desktop\\DNA TFBP\\DNA_TranscriptionFactorBindingPrediction\\models\\model_JSON.json","w") as json_file:
    json_file.write(model_JSON)
model.save_weights("C:\\Users\\konya\\Desktop\\DNA TFBP\\DNA_TranscriptionFactorBindingPrediction\\models\\model_WEIGHTS.h5")
model.save("C:\\Users\\konya\\Desktop\\DNA TFBP\\DNA_TranscriptionFactorBindingPrediction\\models\\model_model.h5")

In [158]:
sub_X_test =  np.array(test_data['sequence'][0:])
encoded_sub_X_test = tokenizer.texts_to_sequences(sub_X_test)
#preds = model.predict(np.array(encoded_sub_X_test))
#print(preds[:10])
preds = model.predict_classes(np.array(encoded_sub_X_test))
print(preds[:10])

sub_df = pd.DataFrame(data=preds,columns={"prediction"})
sub_df.to_csv(path_or_buf="C:\\Users\\konya\\Desktop\\DNA TFBP\\DNA_TranscriptionFactorBindingPrediction\\results\\sub.csv", columns={"prediction"}, header=True, index=True, index_label="id")

[[1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]]


In [162]:
#load saved model and predict
# load json and create model
json_file = open("C:\\Users\\konya\\Desktop\\DNA TFBP\\DNA_TranscriptionFactorBindingPrediction\\models\\model_JSON.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("C:\\Users\\konya\\Desktop\\DNA TFBP\\DNA_TranscriptionFactorBindingPrediction\\models\\model_WEIGHTS.h5")
print("Loaded model from disk")

#predict using loaded model
preds_loaded = loaded_model.predict_classes(np.array(encoded_sub_X_test))
print(preds_loaded[:10])


Loaded model from disk
[[1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]]
